In [30]:
# from transformers import pipeline
# from PIL import Image

# checkpoint = "EdBianchi/vit-fire-detection"
# detector = pipeline(model=checkpoint, task="image-classification")
# img = ("optics.png")

In [33]:
from transformers import AutoProcessor, AutoModelForZeroShotObjectDetection

checkpoint = "google/owlvit-base-patch32"
model = AutoModelForZeroShotObjectDetection.from_pretrained(checkpoint)
processor = AutoProcessor.from_pretrained(checkpoint)

In [93]:
target = Image.open('target.png')
# target = Image.open('target2.png')
query = Image.open('query.png')

In [94]:
import numpy as np

x = np.array(target)
target = np.zeros((x.shape[0], x.shape[1], 3), dtype=np.uint8)
for i in range(x.shape[0]):
    for j in range(x.shape[1]):
        for k in range(x.shape[2]):
            if k != 3:
                target[i, j, k] = x[i, j, k]

x = np.array(query)
query = np.zeros((x.shape[0], x.shape[1], 3), dtype=np.uint8)
for i in range(x.shape[0]):
    for j in range(x.shape[1]):
        for k in range(x.shape[2]):
            if k != 3:
                query[i, j, k] = x[i, j, k]

target = Image.fromarray(target)
query = Image.fromarray(query)

In [95]:
inputs = processor(images=target, query_images=query, return_tensors="pt")

In [ ]:
import torch 
from PIL import ImageDraw

with torch.no_grad():
    outputs = model.image_guided_detection(**inputs)
    target_sizes = torch.tensor([target.size[::-1]])
    results = processor.post_process_image_guided_detection(outputs=outputs, target_sizes=target_sizes)[0]

draw = ImageDraw.Draw(target)

scores = results["scores"].tolist()
boxes = results["boxes"].tolist()

for box, score in zip(boxes, scores):
    xmin, ymin, xmax, ymax = box
    draw.rectangle((xmin, ymin, xmax, ymax), outline="white", width=4)

target